This is the code I used to filter out the relevant columns for the VA educational attainment data. I also used Greg's cleaned up mobility data (and 7 day running averages) to find the maximum mobility change for each county (fips code). I added these to the ed data frame using the fips code as the join key.

-Mariska (3/9/22)

In [16]:
import pandas as pd

#import the original data set
ed_data = pd.read_csv("EAG_Educational_Attainment_by_Locality.csv")

In [17]:
#view the original data set
ed_data.head()

,Locality,locality_fips,state_name,state_fips,rural_or_non_rural_flag,pct_no_high_school_diploma_loc,cnt_no_high_school_diploma_loc,pct_high_school_grad_loc,cnt_total_high_school_grad_loc,pct_total_high_school_grad_loc,...,pct_high_school_grad_Hispanic_loc,pct_bachelors_degree_Caucasian_loc,pct_bachelors_degree_Black_loc,pct_bachelors_degree_Asian_loc,pct_bachelors_degree_Hispanic_loc,pct_poverty_education_no_high_school_loc,pct_poverty_education_high_school_loc,pct_poverty_education_bachelors_or_higher_loc,pct_pursuing_HighEd_All,cnt_pursuing_HighEd_All
0,Surry,51181,Virginia,51,Non-Rural,0.154303,832,0.371662,4560,0.845697,...,0.878,0.225,0.177,0.000,0.293,0.245,0.150,0.052,0.66,39
1,Clarke,51043,Virginia,51,Non-Rural,0.106878,1223,0.297387,10220,0.893122,...,0.751,0.333,0.278,0.149,0.155,0.161,0.102,0.025,0.75,131
2,Mecklenburg,51117,Virginia,51,Rural,0.180888,4507,0.328303,20409,0.819112,...,0.697,0.256,0.107,0.130,0.122,0.322,0.185,0.070,0.65,186
3,Augusta,51015,Virginia,51,Rural,0.129816,7881,0.392281,52828,0.870184,...,0.571,0.236,0.053,0.707,0.114,0.163,0.092,0.030,0.58,413
4,Goochland,51075,Virginia,51,Non-Rural,0.095134,1789,0.242382,17016,0.904866,...,0.830,0.474,0.148,0.761,0.308,0.105,0.083,0.026,0.66,142


In [18]:
#create a new dataframe with selected columns

ed_new = ed_data[['Locality','locality_fips','rural_or_non_rural_flag', 'pct_no_high_school_diploma_loc', 'pct_high_school_grad_loc','pct_Some_College_Associates_loc','pct_bachelors_degree_+_loc']]
ed_new.head(3)

,Locality,locality_fips,rural_or_non_rural_flag,pct_no_high_school_diploma_loc,pct_high_school_grad_loc,pct_Some_College_Associates_loc,pct_bachelors_degree_+_loc
0,Surry,51181,Non-Rural,0.154303,0.371662,0.276335,0.197700
1,Clarke,51043,Non-Rural,0.106878,0.297387,0.296426,0.299310
2,Mecklenburg,51117,Rural,0.180888,0.328303,0.304704,0.186105


In [19]:
#import and view the cleaned mobility data set from Greg
mobility = pd.read_csv('Google_Mob_VA_peakdates_clean.csv')
mobility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21406 entries, 0 to 21405
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  21406 non-null  int64  
 1   date                        21406 non-null  object 
 2   state                       21406 non-null  object 
 3   county                      21406 non-null  object 
 4   census_fips_code            15152 non-null  float64
 5   retail                      14000 non-null  float64
 6   grocery                     13282 non-null  float64
 7   parks                       2863 non-null   float64
 8   transit                     7306 non-null   float64
 9   workplaces                  20373 non-null  float64
 10  residential                 8272 non-null   float64
 11  Non_Residential             21230 non-null  float64
 12  Non_Residential_7d_Rolling  21050 non-null  float64
dtypes: float64(9), int64(1), object

In [26]:
#drop rows with missing fips code
mobility.dropna(subset=['census_fips_code'], inplace=True)

#convert fips code to integer
mobility.census_fips_code = mobility.census_fips_code.astype(int)

In [27]:
#check that fips code type is changed
mobility.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15152 entries, 0 to 21405
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  15152 non-null  int64  
 1   date                        15152 non-null  object 
 2   state                       15152 non-null  object 
 3   county                      15152 non-null  object 
 4   census_fips_code            15152 non-null  int64  
 5   retail                      8674 non-null   float64
 6   grocery                     8407 non-null   float64
 7   parks                       1445 non-null   float64
 8   transit                     4573 non-null   float64
 9   workplaces                  14496 non-null  float64
 10  residential                 5630 non-null   float64
 11  Non_Residential             14998 non-null  float64
 12  Non_Residential_7d_Rolling  14868 non-null  float64
dtypes: float64(8), int64(2), object

In [41]:
#for each fips code, find min Non-Residential 7-d rolling (which is a % change from baseline; negaive values are a decrease)
max_change = mobility.groupby('census_fips_code')[['Non_Residential_7d_Rolling']].min() #[[ stores output as a data frame, rather than series

In [42]:
max_change.head(10)

,Non_Residential_7d_Rolling
census_fips_code,
51001,-24.428571
51003,-44.000000
51005,-28.071429
51007,-39.357143
51009,-26.714286
51011,-32.285714
51013,-59.514286
51015,-30.897619
51017,-68.428571


In [51]:
#add to ed_new

ed_new = ed_new.merge(max_change, how='inner', left_on='locality_fips', right_on='census_fips_code')


In [52]:
#export the new data frame as a CSV file
ed_new.to_csv('VA_ed_data_cleaned.csv')

In [54]:
#check new file
data = pd.read_csv('VA_ed_data_cleaned.csv')
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       94 non-null     int64  
 1   Locality                         94 non-null     object 
 2   locality_fips                    94 non-null     int64  
 3   rural_or_non_rural_flag          94 non-null     object 
 4   pct_no_high_school_diploma_loc   94 non-null     float64
 5   pct_high_school_grad_loc         94 non-null     float64
 6   pct_Some_College_Associates_loc  94 non-null     float64
 7   pct_bachelors_degree_+_loc       94 non-null     float64
 8   Non_Residential_7d_Rolling       94 non-null     float64
dtypes: float64(5), int64(2), object(2)
memory usage: 6.7+ KB
